<a href="https://colab.research.google.com/github/Itamarshahar/3rd_year_project2/blob/main/nlp_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP ex2





## import

In [ ]:
import os
import matplotlib.pyplot as plt

# Define the folder where plots will be saved
output_folder = "answers"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
import numpy as np
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
!pip install evaluate

# subset of categories that we will use
category_dict = {'comp.graphics': 'computer graphics',
                 'rec.sport.baseball': 'baseball',
                 'sci.electronics': 'science, electronics',
                 'talk.politics.guns': 'politics, guns'
                 }


## Helpers funcitons

In [ ]:
def get_data(categories=None, portion=1.):
    """
    Get data for given categories and portion
    :param portion: portion of the data to use
    :return:
    """
    # get data
    from sklearn.datasets import fetch_20newsgroups
    data_train = fetch_20newsgroups(categories=categories, subset='train', remove=('headers', 'footers', 'quotes'),
                                    random_state=21)
    data_test = fetch_20newsgroups(categories=categories, subset='test', remove=('headers', 'footers', 'quotes'),
                                   random_state=21)

    # train
    train_len = int(portion*len(data_train.data))
    x_train = np.array(data_train.data[:train_len])
    y_train = data_train.target[:train_len]
    # remove empty entries
    non_empty = x_train != ""
    x_train, y_train = x_train[non_empty].tolist(), y_train[non_empty].tolist()

    # test
    x_test = np.array(data_test.data)
    y_test = data_test.target
    non_empty = np.array(x_test) != ""
    x_test, y_test = x_test[non_empty].tolist(), y_test[non_empty].tolist()
    return x_train, y_train, x_test, y_test

    """
    Get data for given categories and portion
    :param portion: portion of the data to use
    :return:
    """
    # get data
    from sklearn.datasets import fetch_20newsgroups
    data_train = fetch_20newsgroups(categories=categories, subset='train', remove=('headers', 'footers', 'quotes'),
                                    random_state=21)
    data_test = fetch_20newsgroups(categories=categories, subset='test', remove=('headers', 'footers', 'quotes'),
                                   random_state=21)

    # train
    train_len = int(portion*len(data_train.data))
    x_train = np.array(data_train.data[:train_len])
    y_train = data_train.target[:train_len]
    # remove empty entries
    non_empty = x_train != ""
    x_train, y_train = x_train[non_empty].tolist(), y_train[non_empty].tolist()

    # test
    x_test = np.array(data_test.data)
    y_test = data_test.target
    non_empty = np.array(x_test) != ""
    x_test, y_test = x_test[non_empty].tolist(), y_test[non_empty].tolist()
    return x_train, y_train, x_test, y_test

## Q1 + Q2


In [ ]:
# Model for Q1,2
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(SimpleMLP, self).__init__()
        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.output = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        out = self.output(self.relu(self.hidden(x)))
        return out


In [ ]:
def MLP_classification(portion=1., model=None):
    """
    Perform classification with MLP
    :param portion: portion of the data to use
    :param model: PyTorch model to use
    :return: training losses and validation accuracies
    """
    x_train, y_train, x_test, y_test = get_data(
        categories=category_dict.keys(), portion=portion)

    # Vectorize the data
    vectorizer = TfidfVectorizer(max_features=2000)
    x_train_tfidf = vectorizer.fit_transform(x_train)
    x_test_tfidf = vectorizer.transform(x_test)

    # Convert to tensors
    x_train_tensor = torch.tensor(x_train_tfidf.toarray(), dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    x_test_tensor = torch.tensor(x_test_tfidf.toarray(), dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    # Create Dataset and DataLoader
    train_dataset = torch.utils.data.TensorDataset(x_train_tensor,
                                                   y_train_tensor)
    test_dataset = torch.utils.data.TensorDataset(x_test_tensor, y_test_tensor)

    batch_size = 16
    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                              shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # Define the model
    input_dim = x_train_tfidf.shape[1]  # Should be 2000
    num_classes = len(category_dict)

    if model is None:
        # Default to single-layer perceptron (Q1)
        model = nn.Linear(input_dim, num_classes)
    else:
        model = model  # Use the model passed in (Q2)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # Training loop
    num_epochs = 20
    train_losses = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for x_batch, y_batch in train_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * x_batch.size(0)

        avg_loss = total_loss / len(train_dataset)
        train_losses.append(avg_loss)

        # Evaluate on validation data
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for x_batch, y_batch in test_loader:
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                outputs = model(x_batch)
                _, predicted = torch.max(outputs.data, 1)
                total += y_batch.size(0)
                correct += (predicted == y_batch).sum().item()
        val_accuracy = correct / total
        val_accuracies.append(val_accuracy)

        print(
            f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

    return train_losses, val_accuracies


## Q3

In [ ]:
def transformer_classification(portion=1.):
    import torch
    from transformers import AutoModelForSequenceClassification, AutoTokenizer
    from torch.utils.data import DataLoader
    import evaluate
    from tqdm import tqdm

    class Dataset(torch.utils.data.Dataset):
        """
        Dataset for loading data
        """

        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in
                    self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
            return item

        def __len__(self):
            return len(self.labels)

    def train_epoch(model, data_loader, optimizer, dev='cpu'):
        """
        Perform an epoch of training of the model with the optimizer
        :param model:
        :param data_loader:
        :param optimizer:
        :param dev:
        :return: Average loss over the epoch
        """
        model.train()
        total_loss = 0.
        # iterate over batches
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(dev)
            attention_mask = batch['attention_mask'].to(dev)
            labels = batch['labels'].to(dev)
            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                            labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(data_loader)
        return avg_loss

    def evaluate_model(model, data_loader, dev='cpu', metric=None):
        model.eval()
        if metric is None:
            metric = evaluate.load("accuracy")
        else:
            metric = metric
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(dev)
            attention_mask = batch['attention_mask'].to(dev)
            labels = batch['labels'].to(dev)
            with torch.no_grad():
                outputs = model(input_ids=input_ids,
                                attention_mask=attention_mask)
                logits = outputs.logits
                predictions = torch.argmax(logits, dim=-1)
                metric.add_batch(predictions=predictions.cpu(),
                                 references=labels.cpu())
        return metric.compute()

    x_train, y_train, x_test, y_test = get_data(
        categories=category_dict.keys(), portion=portion)

    # Parameters
    dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    num_labels = len(category_dict)
    epochs = 3
    batch_size = 16
    learning_rate = 5e-5

    # Model, tokenizer, and metric
    model = AutoModelForSequenceClassification.from_pretrained(
        'distilroberta-base', num_labels=num_labels).to(dev)
    tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
    metric = evaluate.load("accuracy")

    # Datasets and DataLoaders
    train_encodings = tokenizer(x_train, truncation=True, padding=True)
    val_encodings = tokenizer(x_test, truncation=True, padding=True)
    train_dataset = Dataset(train_encodings, y_train)
    val_dataset = Dataset(val_encodings, y_test)
    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                              shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Define optimizer
    from transformers import AdamW
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    # Training loop
    train_losses = []
    val_accuracies = []

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        avg_loss = train_epoch(model, train_loader, optimizer, dev=dev)
        train_losses.append(avg_loss)
        val_accuracy = evaluate_model(model, val_loader, dev=dev,
                                      metric=metric)
        accuracy = val_accuracy['accuracy']
        val_accuracies.append(accuracy)
        print(
            f"Average training loss: {avg_loss:.4f}, Validation accuracy: {accuracy:.4f}")

    # Plotting
    plt.figure()
    plt.plot(range(1, epochs + 1), train_losses)
    plt.title(f'Transformer Training Loss - Portion {portion}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.savefig(f'answers/Q3_portion_{portion}_loss.png')
    plt.close()

    plt.figure()
    plt.plot(range(1, epochs + 1), val_accuracies)
    plt.title(f'Transformer Validation Accuracy - Portion {portion}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.savefig(f'answers/Q3_portion_{portion}_accuracy.png')
    plt.close()

    return


## Main run


In [ ]:
portions = [0.1, 0.2, 0.5, 1.]


Q1 - Single Layer MLP - Portion: 0.1
Epoch [1/20], Loss: 1.3822, Val Accuracy: 0.4907
Epoch [2/20], Loss: 1.3567, Val Accuracy: 0.5816
Epoch [3/20], Loss: 1.3336, Val Accuracy: 0.5968
Epoch [4/20], Loss: 1.3108, Val Accuracy: 0.6373
Epoch [5/20], Loss: 1.2891, Val Accuracy: 0.6412
Epoch [6/20], Loss: 1.2670, Val Accuracy: 0.6558
Epoch [7/20], Loss: 1.2458, Val Accuracy: 0.6684
Epoch [8/20], Loss: 1.2251, Val Accuracy: 0.6777
Epoch [9/20], Loss: 1.2044, Val Accuracy: 0.6771
Epoch [10/20], Loss: 1.1844, Val Accuracy: 0.6863
Epoch [11/20], Loss: 1.1647, Val Accuracy: 0.6883
Epoch [12/20], Loss: 1.1452, Val Accuracy: 0.6969
Epoch [13/20], Loss: 1.1262, Val Accuracy: 0.6983
Epoch [14/20], Loss: 1.1075, Val Accuracy: 0.7023
Epoch [15/20], Loss: 1.0893, Val Accuracy: 0.7029
Epoch [16/20], Loss: 1.0712, Val Accuracy: 0.7036
Epoch [17/20], Loss: 1.0534, Val Accuracy: 0.7129
Epoch [18/20], Loss: 1.0360, Val Accuracy: 0.7115
Epoch [19/20], Loss: 1.0189, Val Accuracy: 0.7115
Epoch [20/20], Loss: 

In [ ]:
# Q1 - single layer MLP
for p in portions:
  print(f"\nQ1 - Single Layer MLP - Portion: {p}")
  train_losses, val_accuracies = MLP_classification(portion=p)
  # Plotting
  plt.figure()
  plt.plot(range(1,21), train_losses)
  plt.title(f'Q1 Training Loss - Portion {p}')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.savefig(f'answers/Q1_portion_{p}_loss.png')
  plt.close()

  plt.figure()
  plt.plot(range(1,21), val_accuracies)
  plt.title(f'Q1 Validation Accuracy - Portion {p}')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.savefig(f'answers/Q1_portion_{p}_accuracy.png')
  plt.close()




Q1 - Single Layer MLP - Portion: 0.1
Epoch [1/20], Loss: 1.3825, Val Accuracy: 0.4854
Epoch [2/20], Loss: 1.3560, Val Accuracy: 0.5710
Epoch [3/20], Loss: 1.3327, Val Accuracy: 0.6001
Epoch [4/20], Loss: 1.3102, Val Accuracy: 0.6300
Epoch [5/20], Loss: 1.2882, Val Accuracy: 0.6313
Epoch [6/20], Loss: 1.2666, Val Accuracy: 0.6340
Epoch [7/20], Loss: 1.2449, Val Accuracy: 0.6565
Epoch [8/20], Loss: 1.2240, Val Accuracy: 0.6638
Epoch [9/20], Loss: 1.2036, Val Accuracy: 0.6585
Epoch [10/20], Loss: 1.1835, Val Accuracy: 0.6718
Epoch [11/20], Loss: 1.1638, Val Accuracy: 0.6744
Epoch [12/20], Loss: 1.1442, Val Accuracy: 0.6744
Epoch [13/20], Loss: 1.1249, Val Accuracy: 0.6797
Epoch [14/20], Loss: 1.1063, Val Accuracy: 0.6844
Epoch [15/20], Loss: 1.0879, Val Accuracy: 0.6897
Epoch [16/20], Loss: 1.0701, Val Accuracy: 0.6963
Epoch [17/20], Loss: 1.0522, Val Accuracy: 0.6950
Epoch [18/20], Loss: 1.0348, Val Accuracy: 0.6976
Epoch [19/20], Loss: 1.0177, Val Accuracy: 0.6996
Epoch [20/20], Loss: 

In [ ]:
# Q2 - multi-layer MLP
for p in portions:
  print(f"\nQ2 - Multi-Layer MLP - Portion: {p}")
  input_dim = 2000
  hidden_dim = 500
  num_classes = len(category_dict)
  model = SimpleMLP(input_dim, hidden_dim, num_classes)
  train_losses, val_accuracies = MLP_classification(portion=p, model=model)
  # Plotting
  plt.figure()
  plt.plot(range(1,21), train_losses)
  plt.title(f'Q2 Training Loss - Portion {p}')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.savefig(f'answers/Q2_portion_{p}_loss.png')
  plt.close()

  plt.figure()
  plt.plot(range(1,21), val_accuracies)
  plt.title(f'Q2 Validation Accuracy - Portion {p}')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.savefig(f'answers/Q2_portion_{p}_accuracy.png')
  plt.close()



Q2 - Multi-Layer MLP - Portion: 0.1
Epoch [1/20], Loss: 1.3757, Val Accuracy: 0.5199
Epoch [2/20], Loss: 1.2654, Val Accuracy: 0.7109
Epoch [3/20], Loss: 1.0454, Val Accuracy: 0.7261
Epoch [4/20], Loss: 0.7127, Val Accuracy: 0.7407
Epoch [5/20], Loss: 0.4031, Val Accuracy: 0.7560
Epoch [6/20], Loss: 0.2109, Val Accuracy: 0.7546
Epoch [7/20], Loss: 0.1158, Val Accuracy: 0.7527
Epoch [8/20], Loss: 0.0706, Val Accuracy: 0.7527
Epoch [9/20], Loss: 0.0490, Val Accuracy: 0.7527
Epoch [10/20], Loss: 0.0363, Val Accuracy: 0.7513
Epoch [11/20], Loss: 0.0288, Val Accuracy: 0.7527
Epoch [12/20], Loss: 0.0237, Val Accuracy: 0.7520
Epoch [13/20], Loss: 0.0200, Val Accuracy: 0.7493
Epoch [14/20], Loss: 0.0173, Val Accuracy: 0.7473
Epoch [15/20], Loss: 0.0152, Val Accuracy: 0.7427
Epoch [16/20], Loss: 0.0135, Val Accuracy: 0.7414
Epoch [17/20], Loss: 0.0122, Val Accuracy: 0.7394
Epoch [18/20], Loss: 0.0110, Val Accuracy: 0.7394
Epoch [19/20], Loss: 0.0100, Val Accuracy: 0.7401
Epoch [20/20], Loss: 0

In [ ]:
# Q3 - Transformer
print("\nTransformer results:")
for p in portions[:2]:
  print(f"Portion: {p}")
  transformer_classification(portion=p)


Transformer results:
Portion: 0.1


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3


100%|██████████| 95/95 [00:19<00:00,  4.89it/s]


Average training loss: 1.3122, Validation accuracy: 0.6598
Epoch 2/3


...

Average training loss: 1.0289, Validation accuracy: 0.8097
Epoch 2/3


100%|██████████| 95/95 [00:20<00:00,  4.73it/s]


Average training loss: 0.3239, Validation accuracy: 0.8581
Epoch 3/3


100%|██████████| 95/95 [00:20<00:00,  4.71it/s]


Average training loss: 0.1381, Validation accuracy: 0.8826


In [ ]:
import torch
import transformers

# Log-linear Classifier
def log_linear_params_with_bias(input_dim, output_dim):
    # Weights: input_dim * output_dim
    # Bias: output_dim (one bias for each output neuron)
    weights = input_dim * output_dim
    biases = output_dim
    total_params = weights + biases
    return total_params


# MLP
def mlp_params(input_dim, hidden_dim, output_dim):
    input_to_hidden = input_dim * hidden_dim
    hidden_to_output = hidden_dim * output_dim
    bias_terms = hidden_dim + output_dim
    total_params = input_to_hidden + hidden_to_output + bias_terms
    return total_params

# Transformer
def transformer_params(model_name="distilroberta-base"):
    # Load the transformer model from Hugging Face
    model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Parameter calculations
input_dim = 2000
hidden_dim = 500
output_dim = 4

log_linear = log_linear_params_with_bias(input_dim, output_dim)
mlp = mlp_params(input_dim, hidden_dim, output_dim)
transformer = transformer_params()

# Display the results
print(f"Trainable Parameters:")
print(f"Log-linear Preceptron: {log_linear}")
print(f"MLP: {mlp}")
print(f"Transformer: {transformer}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable Parameters:
Log-linear Preceptron: 8004
MLP: 1002504
Transformer: 82121476


In [ ]:
###################################################
# Exercise 2 - Natural Language Processing 67658  #
###################################################
!pip install transformers
!pip install datasets
!pip install evaluate

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

# subset of categories that we will use
category_dict = {'comp.graphics': 'computer graphics',
                 'rec.sport.baseball': 'baseball',
                 'sci.electronics': 'science, electronics',
                 'talk.politics.guns': 'politics, guns'
                 }

def get_data(categories=None, portion=1.):
    """
    Get data for given categories and portion
    :param portion: portion of the data to use
    :return:
    """
    # get data
    from sklearn.datasets import fetch_20newsgroups
    data_train = fetch_20newsgroups(categories=categories, subset='train', remove=('headers', 'footers', 'quotes'),
                                    random_state=21)
    data_test = fetch_20newsgroups(categories=categories, subset='test', remove=('headers', 'footers', 'quotes'),
                                   random_state=21)

    # train
    train_len = int(portion*len(data_train.data))
    x_train = np.array(data_train.data[:train_len])
    y_train = data_train.target[:train_len]
    # remove empty entries
    non_empty = x_train != ""
    x_train, y_train = x_train[non_empty].tolist(), y_train[non_empty].tolist()

    # test
    x_test = np.array(data_test.data)
    y_test = data_test.target
    non_empty = np.array(x_test) != ""
    x_test, y_test = x_test[non_empty].tolist(), y_test[non_empty].tolist()
    return x_train, y_train, x_test, y_test

# Model for Q2
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Q1,2
def MLP_classification(portion=1., model=None):
    """
    Perform classification with MLP
    :param portion: portion of the data to use
    :param model: PyTorch model to use
    :return: training losses and validation accuracies
    """
    x_train, y_train, x_test, y_test = get_data(categories=category_dict.keys(), portion=portion)

    # Vectorize the data
    vectorizer = TfidfVectorizer(max_features=2000)
    x_train_tfidf = vectorizer.fit_transform(x_train)
    x_test_tfidf = vectorizer.transform(x_test)

    # Convert to tensors
    x_train_tensor = torch.tensor(x_train_tfidf.toarray(), dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    x_test_tensor = torch.tensor(x_test_tfidf.toarray(), dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    # Create Dataset and DataLoader
    train_dataset = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
    test_dataset = torch.utils.data.TensorDataset(x_test_tensor, y_test_tensor)

    batch_size = 16
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # Define the model
    input_dim = x_train_tfidf.shape[1]  # Should be 2000
    num_classes = len(category_dict)

    if model is None:
        # Default to single-layer perceptron (Q1)
        model = nn.Linear(input_dim, num_classes)
    else:
        model = model  # Use the model passed in (Q2)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # Training loop
    num_epochs = 20
    train_losses = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for x_batch, y_batch in train_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * x_batch.size(0)

        avg_loss = total_loss / len(train_dataset)
        train_losses.append(avg_loss)

        # Evaluate on validation data
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for x_batch, y_batch in test_loader:
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                outputs = model(x_batch)
                _, predicted = torch.max(outputs.data, 1)
                total += y_batch.size(0)
                correct += (predicted == y_batch).sum().item()
        val_accuracy = correct / total
        val_accuracies.append(val_accuracy)

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

    return train_losses, val_accuracies

# Q3
def transformer_classification(portion=1.):

    import torch
    from transformers import AutoModelForSequenceClassification, AutoTokenizer
    from torch.utils.data import DataLoader
    import evaluate
    from tqdm import tqdm

    class Dataset(torch.utils.data.Dataset):
        """
        Dataset for loading data
        """
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
            return item

        def __len__(self):
            return len(self.labels)

    def train_epoch(model, data_loader, optimizer, dev='cpu'):
        """
        Perform an epoch of training of the model with the optimizer
        :param model:
        :param data_loader:
        :param optimizer:
        :param dev:
        :return: Average loss over the epoch
        """
        model.train()
        total_loss = 0.
        # iterate over batches
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(dev)
            attention_mask = batch['attention_mask'].to(dev)
            labels = batch['labels'].to(dev)
            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(data_loader)
        return avg_loss

    def evaluate_model(model, data_loader, dev='cpu', metric=None):
        model.eval()
        if metric is None:
            metric = evaluate.load("accuracy")
        else:
            metric = metric
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(dev)
            attention_mask = batch['attention_mask'].to(dev)
            labels = batch['labels'].to(dev)
            with torch.no_grad():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                predictions = torch.argmax(logits, dim=-1)
                metric.add_batch(predictions=predictions.cpu(), references=labels.cpu())
        return metric.compute()

    x_train, y_train, x_test, y_test = get_data(categories=category_dict.keys(), portion=portion)

    # Parameters
    dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    num_labels = len(category_dict)
    epochs = 3
    batch_size = 16
    learning_rate = 5e-5

    # Model, tokenizer, and metric
    model = AutoModelForSequenceClassification.from_pretrained('distilroberta-base', num_labels=num_labels).to(dev)
    tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
    metric = evaluate.load("accuracy")

    # Datasets and DataLoaders
    train_encodings = tokenizer(x_train, truncation=True, padding=True)
    val_encodings = tokenizer(x_test, truncation=True, padding=True)
    train_dataset = Dataset(train_encodings, y_train)
    val_dataset = Dataset(val_encodings, y_test)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Define optimizer
    from transformers import AdamW
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    # Training loop
    train_losses = []
    val_accuracies = []

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        avg_loss = train_epoch(model, train_loader, optimizer, dev=dev)
        train_losses.append(avg_loss)
        val_accuracy = evaluate_model(model, val_loader, dev=dev, metric=metric)
        accuracy = val_accuracy['accuracy']
        val_accuracies.append(accuracy)
        print(f"Average training loss: {avg_loss:.4f}, Validation accuracy: {accuracy:.4f}")

    # Plotting
    plt.figure()
    plt.plot(range(1, epochs+1), train_losses)
    plt.title(f'Transformer Training Loss - Portion {portion}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.savefig(f'Q3_portion_{portion}_loss.png')
    plt.close()

    plt.figure()
    plt.plot(range(1, epochs+1), val_accuracies)
    plt.title(f'Transformer Validation Accuracy - Portion {portion}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.savefig(f'Q3_portion_{portion}_accuracy.png')
    plt.close()

    return

if __name__ == "__main__":

    portions = [0.1, 0.2, 0.5, 1.]

    # # Q1 - single layer MLP
    # for p in portions:
    #     print(f"\nQ1 - Single Layer MLP - Portion: {p}")
    #     train_losses, val_accuracies = MLP_classification(portion=p)
    #     # Plotting
    #     plt.figure()
    #     plt.plot(range(1,21), train_losses)
    #     plt.title(f'Q1 Training Loss - Portion {p}')
    #     plt.xlabel('Epoch')
    #     plt.ylabel('Loss')
    #     plt.savefig(f'Q1_portion_{p}_loss.png')
    #     plt.close()

    #     plt.figure()
    #     plt.plot(range(1,21), val_accuracies)
    #     plt.title(f'Q1 Validation Accuracy - Portion {p}')
    #     plt.xlabel('Epoch')
    #     plt.ylabel('Accuracy')
    #     plt.savefig(f'Q1_portion_{p}_accuracy.png')
    #     plt.close()

    # # Q2 - multi-layer MLP
    # for p in portions:
    #     print(f"\nQ2 - Multi-Layer MLP - Portion: {p}")
    #     input_dim = 2000
    #     hidden_dim = 500
    #     num_classes = len(category_dict)
    #     model = SimpleMLP(input_dim, hidden_dim, num_classes)
    #     train_losses, val_accuracies = MLP_classification(portion=p, model=model)
    #     # Plotting
    #     plt.figure()
    #     plt.plot(range(1,21), train_losses)
    #     plt.title(f'Q2 Training Loss - Portion {p}')
    #     plt.xlabel('Epoch')
    #     plt.ylabel('Loss')
    #     plt.savefig(f'Q2_portion_{p}_loss.png')
    #     plt.close()

    #     plt.figure()
    #     plt.plot(range(1,21), val_accuracies)
    #     plt.title(f'Q2 Validation Accuracy - Portion {p}')
    #     plt.xlabel('Epoch')
    #     plt.ylabel('Accuracy')
    #     plt.savefig(f'Q2_portion_{p}_accuracy.png')
    #     plt.close()

    # Q3 - Transformer
    print("\nTransformer results:")
    for p in portions[:2]:
        print(f"Portion: {p}")
        transformer_classification(portion=p)


Transformer results:
Portion: 0.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.0 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Average training loss: 0.9724, Validation accuracy: 0.8223
Epoch 2/3


100%|██████████| 95/95 [00:20<00:00,  4.57it/s]


Average training loss: 0.3288, Validation accuracy: 0.8528
Epoch 3/3


100%|██████████| 95/95 [00:21<00:00,  4.45it/s]


Average training loss: 0.1505, Validation accuracy: 0.8747


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# NLP ex4


## data loader

In [5]:
import os
import random


POSITIVE_SENTIMENT = 1.
NEGATIVE_SENTIMENT = 0.
NEUTRAL_SENTIMENT = -1.


SENTIMENT_NAMES = {
    POSITIVE_SENTIMENT: "Positive",
    NEUTRAL_SENTIMENT: "Neutral",
    NEGATIVE_SENTIMENT: "Negative"
}
SENTS_PATH = "SOStr.txt"
TREES_PATH = "STree.txt"
DICT_PATH = "dictionary.txt"
LABELS_path = "sentiment_labels.txt"


def get_sentiment_class_from_val(sentiment_val: float):
    if sentiment_val <= 0.4:
        return NEGATIVE_SENTIMENT
    elif sentiment_val >= 0.6:
        return POSITIVE_SENTIMENT
    else:
        return NEUTRAL_SENTIMENT

class SentimentTreeNode(object):
    def __init__(self, text: list, sentiment_val: float, min_token_idx: int, children=[], parent=None):
        self.text = text
        self.sentiment_val = sentiment_val
        self.min_token_idx = min_token_idx
        self.sentiment_class = get_sentiment_class_from_val(sentiment_val)
        self.children = children
        self.parent = parent


class Sentence(object):
    """
    Represents a sentence in sentiment tree bank.
    You can access the sentence text by sent.text
    This will give you a list of tokens (strings) in the order that they appear in the sentence.
    sent.sentiment_class is the coding of the annotated sentiment polarity of the sentence.
    sent.sentiment_val is the exact annotated sentiment value in the range [0,1]
    """
    def __init__(self, sentence_root: SentimentTreeNode):
        self.root = sentence_root
        self.text = sentence_root.text
        self.sentiment_class = sentence_root.sentiment_class
        self.sentiment_val = sentence_root.sentiment_val

    def _get_leaves_recursively(self, cur_root):
        if len(cur_root.children) == 0:
            return [cur_root]
        else:
            cur_leaves = []
            for child in cur_root.children:
                cur_leaves += self._get_leaves_recursively(child)
            return cur_leaves

    def get_leaves(self):
        return self._get_leaves_recursively(self.root)

    def __repr__(self):
        return " ".join(self.text) + " | " + SENTIMENT_NAMES[self.sentiment_class] + " | " + str(self.sentiment_val)


class SentimentTreeBank(object):
    """
    The main object that represents the stanfordSentimentTreeBank dataset. Can be used to access the
    examples and some other utilities.
    """
    def __init__(self, path="stanfordSentimentTreebank", split_ratios=(0.8,0.1,0.1), split_words=True):
        """

        :param path: relative or absolute path to the datset directory
        :param split_ratios: split ratios for train, validation and test. please do not change!
        :param split_words: whether to split tokens with "-" and "/" symbols. please do not change!
        """
        self._base_path = path
        self.split_words = split_words
        sentences = self._read_sentences()
        self.sentences = self._build_dataset(sentences)
        if self.split_words:
            for sent in self.sentences:
                leaves = sent.get_leaves()
                for node in leaves:
                    node_text = node.text
                    splitted = node_text[0].split("-")
                    splitted_final = []
                    for s in splitted:
                        splitted_final.extend(s.split("\\/"))
                    if len(splitted_final) > 1 and all([len(s) > 0 for s in splitted_final]):
                        leaves = [SentimentTreeNode([s], node.sentiment_val,
                                                    min_token_idx=node.min_token_idx, parent=node) for
                                  s in splitted_final]
                        node.text = splitted_final
                        node.children = leaves
                        cur_parent = node.parent
                        while cur_parent != None:
                            cur_parent.text = []
                            for child in cur_parent.children:
                                cur_parent.text.extend(child.text)
                            cur_parent = cur_parent.parent
                sent.text = sent.root.text


        assert len(split_ratios) == 3
        assert sum(split_ratios) == 1
        self.split_ratios = split_ratios



    def _read_sentences(self):
        sentences = []
        with open(os.path.join(self._base_path, SENTS_PATH), "r", encoding="utf-8") as f:
            lines = f.read().split("\n")
            for i, line in enumerate(lines):
                if len(line.strip()) == 0:
                    continue
                line_content = line.strip()
                tokens = line_content.split("|")
                tokens = [t.lower().replace("-lrb-","(").replace("-rrb-", ")") for t in tokens]
                sentences.append(tokens)
        return sentences

    def _build_dataset(self, sentences):
        phrases_dictionary = {}

        # extract phrases
        with open(os.path.join(self._base_path, DICT_PATH), "r", encoding="utf-8") as f:
            lines = f.read().split("\n")[:-1]
            for line in lines:
                phrase, phrase_id = line.strip().split("|")
                phrases_dictionary[phrase.lower().replace("-lrb-","(").replace("-rrb-", ")")] = int(phrase_id)

        # extract labels
        with open(os.path.join(self._base_path, LABELS_path), "r",  encoding="utf-8") as f:
            lines = [l.strip().split("|") for l in f.read().split("\n")[1:-1]]
            labels_dict = {int(l[0]): float(l[1]) for l in lines}


        def get_val_from_phrase(phrase_tokens_list):
            try:
                return labels_dict[phrases_dictionary[" ".join(phrase_tokens_list)]]
            except:
                print("couldn't find key!")

        # load the sentences tree structures
        tree_pointers = []
        with open(os.path.join(self._base_path, TREES_PATH), "r") as f:
            for line in f.readlines():
                sent_pointers = [int(p) for p in line.strip().split("|")]
                tree_pointers.append(sent_pointers)
        assert len(tree_pointers) == len(sentences)

        # create Sentence instances with tree of SentimentTreeNodes
        labeled_sentences = []
        for sent, sent_pointers in zip(sentences, tree_pointers):
            try:

                children_dict = {i: [] for i in range(len(sent_pointers))}
                for i, p in enumerate(sent_pointers):
                    if i < len(sent):
                        node_text = [sent[i]]
                        node = SentimentTreeNode(text=node_text, sentiment_val=get_val_from_phrase(node_text),
                                                 min_token_idx=i)
                    else:
                        children = children_dict[i]
                        children = sorted(children, key= lambda n: n.min_token_idx)
                        node_text = []
                        for child in children:
                            node_text.extend(child.text)
                        node = SentimentTreeNode(text=node_text, sentiment_val=get_val_from_phrase(node_text),
                                                 children=children, min_token_idx=children[0].min_token_idx)
                        for child in children:
                            child.parent = node
                    if p > 0:
                        children_dict[p - 1].append(node)
                    last_node = node
                new_sentence = Sentence(last_node)
                if new_sentence.sentiment_class == NEUTRAL_SENTIMENT:
                    continue
                labeled_sentences.append(new_sentence)
            except Exception as e:
                raise e
                print("couldn't parse sentence!")
                print(sent)
        random.Random(1).shuffle(labeled_sentences) # shuffle but with the same shuffle each time
        return labeled_sentences

    def get_train_set(self):
        """
        :return: list of Sentence instances for the train part of the dataset
        """
        if not hasattr(self, "_train_set"):
            self._train_set = self.sentences[:int(self.split_ratios[0] * len(self.sentences))]
        return self._train_set

    def _extract_all_phrases(self, root):
        phrases = [Sentence(root)] if root.sentiment_class != NEUTRAL_SENTIMENT else []
        if len(root.text) == 1:
            return []
        for child in root.children:
            phrases += self._extract_all_phrases(child)
        return phrases

    def get_train_set_phrases(self):
        """
        :return: list of Sentence instances for the train part of the dataset including all sub-phrases
        """
        if not hasattr(self, "_train_set_phrases"):
            train_set = self.get_train_set()
            train_set_phrases = []
            for sent in train_set:
                train_set_phrases += self._extract_all_phrases(sent.root)
            self._train_set_phrases = train_set_phrases
        return self._train_set_phrases

    def get_test_set(self):
        """
        :return: list of Sentence instances for the test part of the dataset
        """
        if not hasattr(self, "_test_set"):
            begin_index = int(self.split_ratios[0] * len(self.sentences))
            end_index = int(sum(self.split_ratios[:2]) * len(self.sentences))
            self._test_set = self.sentences[begin_index:end_index]
        return self._test_set

    def get_validation_set(self):
        """
        :return: list of Sentence instances for the validation part of the dataset
        """
        if not hasattr(self, "_validation_set"):
            self._validation_set = self.sentences[int(sum(self.split_ratios[:2]) * len(self.sentences)):]
        return self._validation_set

    def get_train_word_counts(self):
        """
        :return: dictionary of all words in the train set with their frequency in the train set.
        """
        if not hasattr(self, "_train_word_counts"):
            word_counts = {}
            for sent in self.get_train_set():
                for word_node in sent.get_leaves():
                    assert len(word_node.text) == 1
                    word_text = word_node.text[0]
                    if word_text in word_counts:
                        word_counts[word_text] += 1
                    else:
                        word_counts[word_text] = 1
            self._train_word_counts = word_counts

        return self._train_word_counts

    def get_word_counts(self):
        """
        :return: dictionary of all words in the dataset with their frequency in the whole dataset.
        """
        if not hasattr(self, "_word_counts"):
            word_counts = {}
            for sent in self.sentences:
                for word_node in sent.get_leaves():
                    assert len(word_node.text) == 1
                    word_text = word_node.text[0]
                    if word_text in word_counts:
                        word_counts[word_text] += 1
                    else:
                        word_counts[word_text] = 1
            self._word_counts = word_counts

        return self._word_counts



def get_negated_polarity_examples(sentences_list, num_examples=None, choose_random=False):
    """
    Returns the indices of the sentences in sentences_list which have subphrase in the second level with
    sentiment polarity different than the whole sentence polarity.
    :param sentences_list: list of Sentence objects
    :param num_examples: number of examples to return, if None all of them are returned
    :param choose_random: relevant only if num_examples is lower than the number of exisitng negated
    polarity examples in sentences_list
    """

    if num_examples is None:
        num_examples = len(sentences_list) # take all possible sentences

    def is_polarized(sent: Sentence):
        if sent.sentiment_class == NEUTRAL_SENTIMENT:
            return False
        else:
            root_polarity = sent.sentiment_class
            for child in sent.root.children:
                if child.sentiment_class == 1 - root_polarity:
                    return True
            return False
    indexed_senteces = list(enumerate(sentences_list))
    negated_sentences = list(filter(lambda s: is_polarized(s[1]), indexed_senteces))
    negated_sentences_indices = [i for i,s in negated_sentences]
    if len(negated_sentences) <= num_examples:
        return negated_sentences_indices
    else:
        if choose_random:
            random.shuffle(negated_sentences_indices)
        return negated_sentences_indices[:num_examples]


def get_sentiment_words(sent: Sentence):
    sent_polarity = sent.sentiment_class
    return [node for node in sent.get_leaves() if node.sentiment_class == sent_polarity]


def get_rare_words_examples(sentences_list, dataset: SentimentTreeBank,
                            num_sentences=50):
    """
    Computes for each sentence in sentences the maximal train frequency of sentiment word, where sentiment
    word is a word which is labeled with either positive or negative sentiment value, and returns the
    indices of the <num_sentences> sentences with lowest value.
    :param sentences_list: list of Sentence objects
    :param dataset: the SentimentTreebank datset object
    :param num_sentences: number of sentences to return
    :return: list of ints representing the indices of the chosen sentences out of the input sentences_list
    """
    word_counts = dataset.get_train_word_counts()

    def get_count(word_node: SentimentTreeNode):
        word_text = word_node.text[0]
        if word_text in word_counts:
            return word_counts[word_text]
        else:
            return 0
    indexed_sentences = list(enumerate(sentences_list))
    indexed_sentences = list(filter(lambda s: len(get_sentiment_words(s[1])) > 0, indexed_sentences))
    indexed_sentences = sorted(indexed_sentences, key= lambda s: max([get_count(node) for node in
                                                                      get_sentiment_words(s[1])]))
    indices = [i for i,s in indexed_sentences]
    return indices[:num_sentences]




if __name__ == '__main__':
    # examples for reading the sentiment dataset
    dataset = SentimentTreeBank()
    # get train set
    print(dataset.get_train_set()[:2])
    print(dataset.get_train_set()[0].sentiment_val)
    # get word counts dictionary
    print(list(dataset.get_word_counts().keys())[:10])


[just too silly and sophomoric to ensnare its target audience . | Negative | 0.29167, there is n't one moment in the film that surprises or delights . | Negative | 0.16667]
0.29167
['just', 'too', 'silly', 'and', 'sophomoric', 'to', 'ensnare', 'its', 'target', 'audience']


## Imports and consts



In [6]:
import collections
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os

from torch import Tensor
from torch.utils.data import DataLoader, TensorDataset, Dataset
import operator
import pickle
import tqdm
import matplotlib.pyplot as plt
from typing import List, Iterable, Tuple, Dict
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_scheduler, AutoTokenizer
import torch
# import torch.optim.AdamW
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from transformers import get_scheduler
import torch.nn as nn
from tqdm import tqdm
import matplotlib.pyplot as plt


In [7]:
SEQ_LEN = 52
W2V_EMBEDDING_DIM = 300

RARE_WORDS = "rare_words"
ONEHOT_AVERAGE = "onehot_average"
W2V_AVERAGE = "w2v_average"
W2V_SEQUENCE = "w2v_sequence"
NEGATIVE_POLARITY = "negative_polarity"
TRAIN = "train"
VAL = "val"
TEST = "test"
ACCURACY = "accuracy"
VALIDATION = "validation"
LOSS = "loss"

## Models

In [8]:
# ------------------------------------ Models ----------------------------------------------------
class LSTM(nn.Module):
    """
    An LSTM for sentiment analysis with architecture as described in the exercise description.
    """
    def __init__(self, embedding_dim, hidden_dim, n_layers, dropout):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim,
                    bidirectional=True, dropout=dropout, num_layers=n_layers, dtype=torch.float64, batch_first=True)
        self.linear1 = nn.Linear(hidden_dim * 2, 1, bias=True, dtype=torch.float64)
    def forward(self, text):
        lstm_out, _ = self.lstm(text)
        lstm_out = torch.cat((lstm_out[:, -1, :self.hidden_dim], lstm_out[:, 0, self.hidden_dim:]), dim=1) # first and last
        return self.linear1(lstm_out)


    def predict(self, text):
        with torch.no_grad():
            return torch.sigmoid(self.forward(text))

class LogLinear(nn.Module):
    """
    general class for the log-linear models for sentiment analysis.
    """
    def __init__(self, embedding_dim):
        super().__init__()
        self.linear1 = nn.Linear(embedding_dim, 1, bias=True, dtype=torch.float64)
        # Add device attribute and move the model to the device
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, x):
        return self.linear1(x)

    def predict(self, x): #TODO validate logits?
        # Move input to the model's device
        x = x.to(self.device)
        with torch.no_grad():
            return torch.sigmoid(self.forward(x))


## Training functions

### Training Helpers

In [25]:
def get_available_device():
    """
    Allows training on GPU if available. Can help with running things faster when a GPU with cuda is
    available but not a most...
    Given a device, one can use module.to(device)
    and criterion.to(device) so that all the computations will be done on the GPU.
    """
    return torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def save_pickle(obj, path):
    with open(path, "wb") as f:
        pickle.dump(obj, f)


def load_pickle(path):
    with open(path, "rb") as f:
        return pickle.load(f)


def save_model(model, path, epoch, optimizer):
    """
    Utility function for saving checkpoint of a model, so training or evaluation can be executed later on.
    :param model: torch module representing the model
    :param optimizer: torch optimizer used for training the module
    :param path: path to save the checkpoint into
    """
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()}, path)


In [26]:
class OnlineDataset(Dataset):
    """
    A pytorch dataset which generates model inputs on the fly from sentences of SentimentTreeBank
    """

    def __init__(self, sent_data, sent_func, sent_func_kwargs):
        """
        :param sent_data: list of sentences from SentimentTreeBank
        :param sent_func: Function which converts a sentence to an input datapoint
        :param sent_func_kwargs: fixed keyword arguments for the state_func
        """
        self.data = sent_data
        self.sent_func = sent_func
        self.sent_func_kwargs = sent_func_kwargs

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sent = self.data[idx]
        sent_emb = self.sent_func(sent, **self.sent_func_kwargs)
        sent_label = sent.sentiment_class
        return sent_emb, sent_label
class DataManager():
    """
    Utility class for handling all data management task. Can be used to get iterators for training and
    evaluation.
    """

    def __init__(self,
                 data_type=ONEHOT_AVERAGE,
                 use_sub_phrases=True,
                 dataset_path="stanfordSentimentTreebank",
                 batch_size=64,
                 embedding_dim=300):
        """
        builds the data manager used for training and evaluation.
        :param data_type: one of ONEHOT_AVERAGE, W2V_AVERAGE and W2V_SEQUENCE
        :param use_sub_phrases: if true, training data will include all sub-phrases plus the full sentences
        :param dataset_path: path to the dataset directory
        :param batch_size: number of examples per batch
        :param embedding_dim: relevant only for the W2V data types.
        """

        # load the dataset
        self.sentiment_dataset = SentimentTreeBank(dataset_path, split_words=True)
        # map data splits to sentences lists
        self.sentences = {} #train
        if use_sub_phrases:
            self.sentences[TRAIN] = self.sentiment_dataset.get_train_set_phrases()
        else:
            self.sentences[TRAIN] = self.sentiment_dataset.get_train_set()

        self.sentences[VAL] = self.sentiment_dataset.get_validation_set()
        self.sentences[TEST] = self.sentiment_dataset.get_test_set()

        self.sentences[NEGATIVE_POLARITY] = [self.sentences[TEST][index] for index in
                                            get_negated_polarity_examples(self.sentences[TEST])]

        self.sentences[RARE_WORDS] = [self.sentences[TEST][index] for index in
                                             get_rare_words_examples(self.sentences[TEST], self.sentiment_dataset)]

        # map data splits to sentence input preperation functions
        words_list = list(self.sentiment_dataset.get_word_counts().keys())
        if data_type == ONEHOT_AVERAGE:
            self.sent_func = average_one_hots
            self.sent_func_kwargs = {"word_to_ind": get_word_to_ind(words_list)}
        elif data_type == W2V_SEQUENCE:
            self.sent_func = sentence_to_embedding

            self.sent_func_kwargs = {"seq_len": SEQ_LEN,
                                     "word_to_vec": create_or_load_slim_w2v(words_list),
                                     "embedding_dim": embedding_dim
                                     }
        elif data_type == W2V_AVERAGE:
            self.sent_func = get_w2v_average
            words_list = list(self.sentiment_dataset.get_word_counts().keys())
            self.sent_func_kwargs = {"word_to_vec": create_or_load_slim_w2v(words_list),
                                     "embedding_dim": embedding_dim
                                     }
        else:
            raise ValueError("invalid data_type: {}".format(data_type))
        # map data splits to torch datasets and iterators
        self.torch_datasets = {k: OnlineDataset(sentences, self.sent_func, self.sent_func_kwargs) for
                               k, sentences in self.sentences.items()}
        self.torch_iterators = {k: DataLoader(dataset, batch_size=batch_size, shuffle=k == TRAIN)
                                for k, dataset in self.torch_datasets.items()}
    def get_torch_iterator(self, data_subset=TRAIN):
        """
        :param data_subset: one of TRAIN VAL and TEST
        :return: torch batches iterator for this part of the datset
        """
        return self.torch_iterators[data_subset]

    def get_labels(self, data_subset=TRAIN):
        """
        :param data_subset: one of TRAIN VAL and TEST
        :return: numpy array with the labels of the requested part of the datset in the same order of the
        examples.
        """
        return np.array([sent.sentiment_class for sent in self.sentences[data_subset]])

    def get_input_shape(self):
        """
        :return: the shape of a single example from this dataset (only of x, ignoring y the label).
        """
        return self.torch_datasets[TRAIN][0][0].shape


In [27]:

def load_word2vec():
    """ Load Word2Vec Vectors
        Return:
            wv_from_bin: All 3 million embeddings, each lengh 300
    """
    import gensim.downloader as api
    wv_from_bin = api.load("word2vec-google-news-300")
    vocab = list(wv_from_bin.key_to_index.keys())
    print(wv_from_bin.key_to_index[vocab[0]])
    print("Loaded vocab size %i" % len(vocab))
    return wv_from_bin
def create_or_load_slim_w2v(words_list, cache_w2v=True):
    """
    returns word2vec dict only for words which appear in the dataset.
    :param words_list: list of words to use for the w2v dict
    :param cache_w2v: whether to save locally the small w2v dictionary
    :return: dictionary which maps the known words to their vectors
    """
    w2v_path = "w2v_dict.pkl"
    if not os.path.exists(w2v_path):
        full_w2v = load_word2vec()
        w2v_emb_dict = {k: full_w2v[k] for k in words_list if k in full_w2v}
        if cache_w2v:
            save_pickle(w2v_emb_dict, w2v_path)
    else:
        w2v_emb_dict = load_pickle(w2v_path)
    return w2v_emb_dict
def average_one_hots(sent, word_to_ind):
    """
    This method gets a sentence and a mapping between words to indices, and returns the average
    one-hot embedding of the tokens in the sentence.
    :param sent: a sentence object.
    :param word_to_ind: a mapping between words to indices
    :return: The average embedding vector as a numpy ndarray.
    """
    total_word = len(sent.text)
    res = np.zeros(len(word_to_ind))
    word_count = collections.Counter(sent.text)

    for word, count in word_count.items():
        if word in word_to_ind:
            res[word_to_ind[word]] += count / total_word

    return res
    # return np.mean([word_to_vec[word] for word in sent.text if word in word_to_vec], axis=0).reshape(-1)

    # return
def get_one_hot(size, ind) -> np.ndarray:
    """
    this method returns a one-hot vector of the given size, where the 1 is placed in the ind entry.
    :param size: the size of the vector
    :param ind: the entry index to turn to 1
    :return: numpy ndarray which represents the one-hot vector
    """
    # first index is 0
    res = np.zeros((size, 1), dtype=np.float32)
    res[ind] = 1
    return res

def get_w2v_average(sent:Sentence, word_to_vec:Dict[str, int], embedding_dim:int=300) -> np.ndarray:
    """
    This method gets a sentence and returns the average word embedding of the words consisting
    the sentence.
    :param sent: the sentence object
    :param word_to_vec: a dictionary mapping words to their vector embeddings
    :param embedding_dim: the dimension of the word embedding vectors
    :return The average embedding vector as numpy ndarray.
    """
    dict_1 = collections.Counter(sent.text)
    res = np.zeros(embedding_dim)
    total_word = len(sent.text)
    for word, count in dict_1.items():
        if word in word_to_vec:
            res += (word_to_vec[word] * count) / total_word
    return res

def get_word_to_ind(words_list:List[str]) -> dict[str, int]:
    """
    this function gets a list of words, and returns a mapping between
    words to their index.
    :param words_list: a list of words
    :return: the dictionary mapping words to the index
    """
    return {word: i for i, word in enumerate(words_list)}

def sentence_to_embedding(sent:Sentence, word_to_vec:Dict[str, int], seq_len:int, embedding_dim:int =300):
    """
    this method gets a sentence and a word to vector mapping, and returns a list containing the
    words embeddings of the tokens in the sentence.
    :param sent: a sentence object
    :param word_to_vec: a word to vector mapping.
    :param seq_len: the fixed length for which the sentence will be mapped to.
    :param embedding_dim: the dimension of the w2v embedding
    :return: numpy ndarray of shape (seq_len, embedding_dim) with the representation of the sentence
    """
    # if not embedding_dim:
    #     embedding_dim = 300
    res = np.zeros((SEQ_LEN, embedding_dim))
    text = sent.text
    for i, word in enumerate(text):
        if i >= seq_len:
            break
        if word in word_to_vec:
            res[i] = word_to_vec[word]
    return res



# ------------------------- training functions -------------

# ------------------------ Print functions ------------------------------------

def print_test_results(test_loss, test_accuracy, model_name="simple Log-Linear model"):
    """
    This function prints the results of the testing process.
    """
    print(f'{model_name.title()} {TEST} {LOSS}: {test_loss:.4f}, {TEST} {ACCURACY}: {test_accuracy:.4f}')
def plot_training_process(num_epochs: int,
                          train_losses: List[float],
                          train_accuracies: List[float],
                          val_losses: List[float],
                          val_accuracies: List[float],
                          model_name="simple Log-Linear model",
                          save_plots=True):
    """
    This function plots the training process of the model.
    """

    train_and_validation = f"{TRAIN} & {VALIDATION}"
    def _save_or_show(plt, save_plots:bool, model_name:str, type=LOSS):
        if save_plots:
            plt.savefig(f"{model_name}_{type}.png")
        else:
            plt.show()

    # Plotting the epoch loss
    plt.plot(range(1, num_epochs + 1), train_losses, label=f'{TRAIN.title()} Loss')
    plt.plot(range(1, num_epochs + 1), val_losses, label=f'{VALIDATION.title()} Loss')
    plt.xlabel('Epochs')
    plt.ylabel(f'{LOSS.title()}')
    plt.title(f"{model_name.title()} {train_and_validation.title()} Losses")
    plt.legend()
    _save_or_show(plt, save_plots, model_name)

    # Plotting the Accuracies
    plt.plot(range(1, num_epochs + 1), train_accuracies,
             label=f'{TRAIN.title()} {ACCURACY.title()}')
    plt.plot(range(1, num_epochs + 1), val_accuracies,
             label=f'{VALIDATION.title()} {ACCURACY.title()}')
    plt.xlabel('Epochs'.title())
    plt.ylabel(f'{ACCURACY.title()}')
    plt.title(f"{model_name.title()} {train_and_validation.title()} Accuracies")
    plt.legend()

    _save_or_show(plt, save_plots, model_name, ACCURACY)
def print_train_results(epoch,
                        num_epochs,
                        epoch_train_loss,
                        epoch_train_accuracy,
                        epoch_val_loss,
                        epoch_val_accuracy, model_name="simple Log-Linear model"):
    """
    This function prints the results of the training process.
    """

    print(f'{model_name.title()} Epoch {epoch + 1}/{num_epochs}, '
          f'{model_name.title()} {TRAIN.title()} {LOSS.title()}: {epoch_train_loss:.4f}, {TRAIN.title()} {ACCURACY.title()}: {epoch_train_accuracy:.4f}, '
          f'{model_name.title()} {VALIDATION.title()} {LOSS.title()}: {epoch_val_loss:.4f}, {VALIDATION.title()} {ACCURACY.title()}: {epoch_val_accuracy:.4f}\n')



### Transformer

In [28]:
def train_transformer_with_distilroberta():
    """
    Trains and evaluates a Transformer model (DistilRoBERTa) for the sentiment analysis task.
    """
    # Load the dataset using DataManager
    data_manager = DataManager(W2V_SEQUENCE, embedding_dim=300, batch_size=64)

    # Load the tokenizer and model
    tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')
    model = DistilBertForSequenceClassification.from_pretrained('distilroberta-base', num_labels=1).to(get_available_device())

    # Prepare DataLoader for Transformer
    train_sentences = data_manager.sentences[TRAIN]
    val_sentences = data_manager.sentences[VAL]
    test_sentences = data_manager.sentences[TEST]

    def encode_sentences(sentences, tokenizer, max_length=52):
        """Helper function to tokenize sentences."""
        texts = [" ".join(sent.text) for sent in sentences]
        labels = [sent.sentiment_class for sent in sentences]
        encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
        return TensorDataset(encodings['input_ids'], encodings['attention_mask'], torch.tensor(labels))

    train_data = encode_sentences(train_sentences, tokenizer)
    val_data = encode_sentences(val_sentences, tokenizer)
    test_data = encode_sentences(test_sentences, tokenizer)

    train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=64)
    test_loader = DataLoader(test_data, batch_size=64)

    # Optimizer and Scheduler
    optimizer = AdamW(model.parameters(), lr=1e-5)
    scheduler = get_scheduler("linear", optimizer, num_warmup_steps=0, num_training_steps=2 * len(train_loader))

    criterion = nn.BCEWithLogitsLoss().to(get_available_device())

    # Training Loop
    train_loss, train_acc = [], []
    val_loss, val_acc = [], []

    for epoch in range(2):  # 2 epochs as specified
        model.train()
        epoch_loss, epoch_acc = 0, 0
        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(get_available_device()) for x in batch]

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            epoch_loss += loss.item()
            epoch_acc += binary_accuracy(logits, labels).cpu().item()  # Move tensor to CPU

        train_loss.append(epoch_loss / len(train_loader))
        train_acc.append(epoch_acc / len(train_loader))

        # Validation
        model.eval()
        val_epoch_loss, val_epoch_acc = 0, 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids, attention_mask, labels = [x.to(get_available_device()) for x in batch]

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                logits = outputs.logits

                val_epoch_loss += loss.item()
                val_epoch_acc += binary_accuracy(logits, labels).cpu().item()  # Move tensor to CPU

        val_loss.append(val_epoch_loss / len(val_loader))
        val_acc.append(val_epoch_acc / len(val_loader))

        print_train_results(epoch, 2, train_loss[-1], train_acc[-1], val_loss[-1], val_acc[-1], model_name="Transformer")

    # Plot results
    plot_training_process(2, train_loss, train_acc, val_loss, val_acc, model_name="Transformer")

    # Test Set Evaluation
    model.eval()
    test_epoch_loss, test_epoch_acc = 0, 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = [x.to(get_available_device()) for x in batch]

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            test_epoch_loss += loss.item()
            test_epoch_acc += binary_accuracy(logits, labels).cpu().item()  # Move tensor to CPU

    print_test_results(test_epoch_loss / len(test_loader), test_epoch_acc / len(test_loader), "Transformer")

    return model

### Transformer v2.0

#### helpers

In [43]:
def prepare_dataset(data_iter, tokenizer, batch_size, max_len):
    dataset = []
    for inputs, labels in data_iter:
        # Ensure inputs are strings
        if not isinstance(inputs[0], str):
            inputs = [str(input_) for input_ in inputs]

        # Tokenize inputs
        encodings = tokenizer(inputs, truncation=True, padding=True, max_length=max_len, return_tensors="pt")

        # Combine tokenized inputs and labels into a dictionary
        for i in range(len(inputs)):
            dataset.append({
                "input_ids": encodings["input_ids"][i],
                "attention_mask": encodings["attention_mask"][i],
                "labels": torch.tensor(labels[i], dtype=torch.long)
            })

    return dataset

In [34]:
class CustomDataset(torch.utils.data.Dataset):
    """
    Custom Dataset for tokenized data and labels.
    """
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

        # Ensure encodings and labels are aligned
        if len(self.labels) != len(self.encodings['input_ids']):
            raise ValueError(
                f"Mismatch between encodings and labels: {len(self.encodings['input_ids'])} vs {len(self.labels)}"
            )

    def __getitem__(self, idx):
        # Ensure valid indexing
        if idx >= len(self.labels):
            raise IndexError(f"Index {idx} out of bounds for dataset of size {len(self.labels)}")

        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [36]:
def train_transformer_with_distilroberta_v2(data_manager, n_epochs=2, lr=5e-6, weight_decay=0.0, batch_size=16, max_len=52):
    """
    Train and evaluate a Transformer model (distilroberta-base) for sentiment classification.

    :param data_manager: DataManager object handling dataset and iterators.
    :param n_epochs: Number of epochs for training.
    :param lr: Learning rate for the optimizer.
    :param weight_decay: Weight decay for L2 regularization.
    :param batch_size: Batch size for DataLoader.
    :param max_len: Maximum sequence length for tokenization.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


    # Load model and tokenizer
    tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')
    model = RobertaForSequenceClassification.from_pretrained('distilroberta-base', num_labels=1).to(device)

    # Define optimizer and loss function
    optimizer = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.BCEWithLogitsLoss().to(device)

    # Prepare datasets
    train_loader = prepare_dataset(data_manager.get_torch_iterator(TRAIN), tokenizer, batch_size, max_len)
    val_loader = prepare_dataset(data_manager.get_torch_iterator(VAL), tokenizer, batch_size, max_len)

    # Training and validation
    train_losses, val_losses, val_accuracies = [], [], []
    best_val_accuracy = 0.0

    for epoch in range(n_epochs):
        print(f"Epoch {epoch + 1}/{n_epochs}")

        # Training
        train_loss = train_epoch_transformer_with_distilroberta_v2(model, train_loader, optimizer, criterion, device)
        train_losses.append(train_loss)

        # Validation
        val_loss, val_accuracy = evaluate_model(model, val_loader, criterion, device)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

        # Save the best model
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), f"best_model_epoch_{epoch + 1}.pt")

    # Evaluate on special cases
    evaluate_special_cases(model, data_manager, tokenizer, device, max_len)

    # Plot training progress
    plot_training_process(n_epochs, train_losses, val_losses, val_accuracies)

In [35]:

def train_epoch_transformer_with_distilroberta_v2(model, data_loader, optimizer, criterion, device):
    """
    Perform an epoch of training for the model.
    """
    model.train()
    total_loss = 0.0
    for batch in tqdm(data_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits.squeeze()
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(data_loader)




In [31]:
def evaluate_model(model, data_loader, criterion, device):
    """
    Evaluate the model on validation or test data.
    """
    model.eval()
    total_loss, total_accuracy = 0.0, 0.0

    with torch.no_grad():
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device).squeeze(-1)  # Ensure labels have correct shape [batch_size]

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits.squeeze()
            loss = criterion(logits, labels)
            total_loss += loss.item()

            predictions = torch.round(torch.sigmoid(logits))
            total_accuracy += (predictions == labels).float().mean().item()

    avg_loss = total_loss / len(data_loader)
    avg_accuracy = total_accuracy / len(data_loader)
    return avg_loss, avg_accuracy

In [32]:
def evaluate_special_cases(model, data_manager, tokenizer, device, max_len):
    """
    Evaluate model on special subsets (NEGATIVE_POLARITY and RARE_WORDS).
    """
    print("________________________________________________________________________________________________")
    print("NEGS: ")
    neg_loader = prepare_dataset(data_manager.get_torch_iterator(NEGATIVE_POLARITY), tokenizer, 16, max_len)
    predictions_neg, labels_neg = get_predictions_for_transformer(model, neg_loader, device)
    neg_accuracy = binary_accuracy(predictions_neg, labels_neg)
    neg_loss = nn.BCEWithLogitsLoss()(predictions_neg.squeeze(), labels_neg)
    print_test_results(neg_loss, neg_accuracy, "NEGATIVE_POLARITY")

    print("________________________________________________________________________________________________")
    print("RARE: ")
    rare_loader = prepare_dataset(data_manager.get_torch_iterator(RARE_WORDS), tokenizer, 16, max_len)
    predictions_rare, labels_rare = get_predictions_for_transformer(model, rare_loader, device)
    rare_accuracy = binary_accuracy(predictions_rare, labels_rare)
    rare_loss = nn.BCEWithLogitsLoss()(predictions_rare.squeeze(), labels_rare)
    print_test_results(rare_loss, rare_accuracy, "RARE_WORDS")

def get_predictions_for_transformer(model, data_loader, device):
    """
    Get predictions for a given dataset.
    """
    model.eval()
    predictions, labels = [], []

    with torch.no_grad():
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            batch_labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits.squeeze()
            predictions.append(logits)
            labels.append(batch_labels)

### Other models

In [17]:
def binary_accuracy(preds, y):
    """
    This method returns the accuracy of the predictions, relative to the labels.
    :param preds: a tensor of predictions
    :param y: a tensor of true labels
    :return: scalar value - (<number of accurate predictions> / <number of examples>)
    """
    # Apply sigmoid to predictions
    preds = torch.sigmoid(preds)

    # Round predictions to 0/1
    pred_labels = torch.round(preds)

    # Cast to float, ensure same datatype
    y = y.float()

    # Move y to the same device as pred_labels
    y = y.to(pred_labels.device)

    # Compute number of accurate predictions
    correct = (pred_labels == y).float().sum()

    # Divide by number of examples for accuracy
    acc = correct / len(y)

    return acc

    # preds = torch.sigmoid(preds)
    # # Convert predictions to binary values (0 or 1)
    # preds_binary = (preds >= 0.5).float()
    #
    # # Count the number of accurate predictions
    # correct_predictions = (preds_binary == y).sum().item()
    #
    # # Calculate accuracy
    # accuracy = correct_predictions / len(y)
    #
    # return accuracy
def train_epoch(model, data_iterator, optimizer, criterion) -> Tuple[float, float]:
    """
    This method operates one epoch (pass over the whole train set) of training
    of the given model,
    and returns the accuracy and loss for this epoch
    :param model: the model we're currently training
    :param data_iterator: an iterator, iterating over the training data for the model.
    :param optimizer: the optimizer object for the training process.
    :param criterion: the criterion object for the training process.
    """
    # batch_counter = 0
    # epoch_loss = 0
    # epoch_accuracy = 0

    model.train()  # set the model to training mode
    for train_features, train_labels in data_iterator:
        train_features = train_features.to(get_available_device())
        train_labels = train_labels.to(get_available_device())
        #start_time = time.time()
        outputs = model(train_features).to(get_available_device())
        optimizer.zero_grad()
        # train_features = train_features.squeeze(-1)
        # outputs = model(train_features).squeeze(-1)
        train_labels = train_labels.unsqueeze(-1)
        loss = criterion(outputs, train_labels).to(get_available_device())
        loss.backward()
        optimizer.step()
    model.train(False)

        # epoch_loss += loss.item()  # assuming loss is a scalar tensor
        # epoch_accuracy += binary_accuracy(torch.sigmoid(outputs), train_labels)
        #
        # batch_counter += 1
        # print("BATCH " + str(batch_counter) + " ENDED. loss, accuracy:", epoch_loss, epoch_accuracy, time.time() - start_time)

    # average_loss = epoch_loss / len(data_iterator)
    # average_accuracy = epoch_accuracy / len(data_iterator)
    #
    # return average_loss, average_accuracy


def get_predictions_for_data(model, data_iter):
    """
    This function should iterate over all batches of examples from data_iter and return all of the models
    predictions as a numpy ndarray or torch tensor (or list if you prefer). the prediction should be in the
    same order of the examples returned by data_iter.
    :param model: one of the models you implemented in the exercise
    :param data_iter: torch iterator as given by the DataManager
    :return:
    """
    res = []
    for i, batch in enumerate(data_iter):
        inputs, labels = batch
        inputs = inputs.to(model.device) # move input to the model's device
        outputs = model.predict(inputs)
        if i == 0:
            res = outputs
        else:
            res = torch.cat((res, outputs), dim=0)
    return res

def train_model(model, data_manager, n_epochs=20, lr=0.01, weight_decay=0., model_name="Simple Log-Linear") -> None:
    """
    Runs the full training procedure for the given model. The optimization should be done using the Adam
    optimizer with all parameters but learning rate and weight decay set to default.
    :param model: module of one of the models implemented in the exercise
    :param data_manager: the DataManager object
    :param n_epochs: number of times to go over the whole training set
    :param lr: learning rate to be used for optimization
    :param weight_decay: parameter for l2 regularization
    """
    train_loss = []
    train_accuracy = []
    validation_loss = []
    validation_accuracy = []
    model_parameters:Iterable[Tensor] = model.parameters()
    optimizer = optim.Adam(model_parameters, lr=lr, weight_decay=weight_decay)
    criterion = nn.BCEWithLogitsLoss().to(get_available_device())
    data_iterator = data_manager.get_torch_iterator(TRAIN)
    for epoch in range(n_epochs):
        # train the model for one epoch
        # evaluate the model on the validation set
        # print the results for the epoch
        # save the model if it got the best validation accuracy until now
        # cur_train_loss, cur_train_accuracy = train_epoch(model=model,
        #                                                 data_iterator=data_manager.get_torch_iterator(data_subset=TRAIN),
        #                                                 optimizer=optimizer,
        #                                                 criterion=criterion)
        train_epoch(model=model,
                    data_iterator=data_iterator,
                    optimizer=optimizer,
                    criterion=criterion.to(get_available_device()))
        cur_train_loss, cur_train_accuracy = evaluate(model=model, data_iterator=data_iterator, criterion=criterion)
        train_loss.append(cur_train_loss)
        train_accuracy.append(cur_train_accuracy)
        cur_val_loss, cur_val_accuracy = evaluate(model,
                                                  data_manager.get_torch_iterator(VAL),
                                                  criterion.to(get_available_device()))

        validation_loss.append(cur_val_loss)
        validation_accuracy.append(cur_val_accuracy)
        print_train_results(epoch, n_epochs, cur_train_loss, cur_train_accuracy, cur_val_loss, cur_val_accuracy, model_name=model_name)
    plot_training_process(n_epochs, train_loss, train_accuracy, validation_loss, validation_accuracy, model_name)
def train_log_linear_with_one_hot():
    """
    Here comes your code for training and evaluation of the log linear model with one hot representation.
    """
    data_manager = DataManager()#batch_size=64)
                                # data_type=ONEHOT_AVERAGE)
                                # use_sub_phrases=True,
                                # dataset_path="stanfordSentimentTreebank",
                                # batch_size=50,
                                # embedding_dim=None)
    one_hot_size = data_manager.get_input_shape()[0]
    linear_model = LogLinear(one_hot_size).to(get_available_device())
    train_model(model=linear_model,
                data_manager=data_manager,
                n_epochs=20, weight_decay=0.001, lr=0.01)
    predictions = get_predictions_for_data(linear_model, data_manager.get_torch_iterator(TEST))
    lables = data_manager.get_labels(TEST)
    test_accuracy = binary_accuracy(predictions.squeeze().round(), torch.tensor(lables))
    test_loss = nn.BCEWithLogitsLoss()(predictions.squeeze(), torch.tensor(lables))
    print_test_results(test_loss, test_accuracy)

    print("________________________________________________________________________________________________")
    print("NEGS: ")

    predictions_neg = get_predictions_for_data(linear_model, data_manager.get_torch_iterator(NEGATIVE_POLARITY))
    lables_neg = data_manager.get_labels(NEGATIVE_POLARITY)
    test_accuracy_neg = binary_accuracy(predictions_neg.squeeze().round(), torch.tensor(lables_neg))
    test_loss_neg = nn.BCEWithLogitsLoss()(predictions_neg.squeeze(), torch.tensor(lables_neg))
    print_test_results(test_loss_neg, test_accuracy_neg)

    print("________________________________________________________________________________________________")
    print("RARE: ")

    predictions_rare = get_predictions_for_data(linear_model, data_manager.get_torch_iterator(RARE_WORDS))
    lables_rare = data_manager.get_labels(RARE_WORDS)
    test_accuracy_rare = binary_accuracy(predictions_rare.squeeze().round(), torch.tensor(lables_rare))
    test_loss_rare = nn.BCEWithLogitsLoss()(predictions_rare.squeeze(), torch.tensor(lables_rare))
    print_test_results(test_loss_rare, test_accuracy_rare)

def train_log_linear_with_w2v():
    """
    Here comes your code for training and evaluation of the log linear model with word embeddings
    representation.
    """
    data_manager = DataManager(data_type=W2V_AVERAGE,
                               embedding_dim=300,
                               batch_size=64)
    linear_model = LogLinear(300).to(get_available_device())
    train_model(model=linear_model,
                data_manager=data_manager,
                n_epochs=20,
                model_name="log linear with W2V", weight_decay=0.001, lr=0.01)
    predictions = get_predictions_for_data(linear_model,
                                           data_manager.get_torch_iterator(
                                               TEST))
    lables = data_manager.get_labels(TEST)
    lables = torch.tensor(lables, device=predictions.device)

    test_accuracy = binary_accuracy(predictions.squeeze().round(),
                                    torch.tensor(lables))
    test_loss = nn.BCEWithLogitsLoss()(predictions.squeeze(),
                                       torch.tensor(lables))
    print_test_results(test_loss, test_accuracy, "log linear with W2V")

    print("________________________________________________________________________________________________")
    print("NEGS: ")

    predictions_neg = get_predictions_for_data(linear_model, data_manager.get_torch_iterator(NEGATIVE_POLARITY))
    lables_neg = data_manager.get_labels(NEGATIVE_POLARITY)
    test_accuracy_neg = binary_accuracy(predictions_neg.squeeze().round(), torch.tensor(lables_neg))
    test_loss_neg = nn.BCEWithLogitsLoss()(predictions_neg.squeeze(), torch.tensor(lables_neg))
    print_test_results(test_loss_neg, test_accuracy_neg)

    print("________________________________________________________________________________________________")
    print("RARE: ")

    predictions_rare = get_predictions_for_data(linear_model, data_manager.get_torch_iterator(RARE_WORDS))
    lables_rare = data_manager.get_labels(RARE_WORDS)
    test_accuracy_rare = binary_accuracy(predictions_rare.squeeze().round(), torch.tensor(lables_rare))
    test_loss_rare = nn.BCEWithLogitsLoss()(predictions_rare.squeeze(), torch.tensor(lables_rare))
    print_test_results(test_loss_rare, test_accuracy_rare)

    return


def train_lstm_with_w2v():
    """
    Here comes your code for training and evaluation of the LSTM model.
    """
    data_manager = DataManager(W2V_SEQUENCE, embedding_dim=300, batch_size=64)
    lstm_model = LSTM(embedding_dim=300, hidden_dim=100, n_layers=1, dropout=0.5).to(get_available_device())

    train_model(model=lstm_model, data_manager=data_manager, n_epochs=4, model_name="LSTM model with W2V", lr=0.001, weight_decay=0.0001)
    predictions = get_predictions_for_data(lstm_model,
                                           data_manager.get_torch_iterator(
                                               TEST))
    lables = data_manager.get_labels(TEST)
    test_accuracy = binary_accuracy(predictions.squeeze().round(),
                                    torch.tensor(lables))
    test_loss = nn.BCEWithLogitsLoss()(predictions.squeeze(),
                                       torch.tensor(lables))
    print_test_results(test_loss, test_accuracy, "LSTM model with W2V")

    print("________________________________________________________________________________________________")
    print("NEGS: ")

    predictions_neg = get_predictions_for_data(lstm_model, data_manager.get_torch_iterator(NEGATIVE_POLARITY))
    lables_neg = data_manager.get_labels(NEGATIVE_POLARITY)
    test_accuracy_neg = binary_accuracy(predictions_neg.squeeze().round(), torch.tensor(lables_neg))
    test_loss_neg = nn.BCEWithLogitsLoss()(predictions_neg.squeeze(), torch.tensor(lables_neg))
    print_test_results(test_loss_neg, test_accuracy_neg)

    print("________________________________________________________________________________________________")
    print("RARE: ")

    predictions_rare = get_predictions_for_data(lstm_model, data_manager.get_torch_iterator(RARE_WORDS))
    lables_rare = data_manager.get_labels(RARE_WORDS)
    test_accuracy_rare = binary_accuracy(predictions_rare.squeeze().round(), torch.tensor(lables_rare))
    test_loss_rare = nn.BCEWithLogitsLoss()(predictions_rare.squeeze(), torch.tensor(lables_rare))
    print_test_results(test_loss_rare, test_accuracy_rare)
    return


### Evaluate

In [18]:
def evaluate(model, data_iterator, criterion):
    """
    Evaluate the model on the validation set.
    """
    model.eval()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for batch in data_iterator:
            validation_features, validation_labels = batch
            # validation_features = validation_features.squeeze(-1)

            # Move validation_features and validation_labels to the correct device
            validation_features = validation_features.to(get_available_device())
            validation_labels = validation_labels.to(get_available_device()) # Move labels to device

            outputs = model(validation_features)
            validation_labels = validation_labels.unsqueeze(-1)
            loss = criterion(outputs, validation_labels) # Remove .to() here

            total_loss += loss.item() * validation_features.size(0)
            predicted = (outputs > 0.5).int()
            total_correct += (predicted == validation_labels).sum().item()
            total_samples += validation_features.size(0)

    avg_loss = total_loss / total_samples
    accuracy = total_correct / total_samples

    return avg_loss, accuracy

## Main run

In [ ]:
init_data_manager = False

In [44]:

if __name__ == '__main__':
    # train_log_linear_with_one_hot()
    #train_log_linear_with_w2v()
    # train_lstm_with_w2v()
    # train_transformer_with_distilroberta()
    # data_manager = DataManager(W2V_SEQUENCE, embedding_dim=300, batch_size=64)
    print(f"{init_data_manager=}")
    if init_data_manager:
      device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
      data_manager = DataManager(W2V_SEQUENCE, embedding_dim=300, batch_size=64)
      init_data_manager = False
    train_transformer_with_distilroberta_v2(data_manager)

init_data_manager=False


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2


  0%|          | 0/2 [00:00<?, ?it/s]


TypeError: list indices must be integers or slices, not str

## Debugging

In [ ]:
data_iter = data_manager.get_torch_iterator(TRAIN)

for i, (inputs, labels) in enumerate(data_iter):
    print(f"Batch {i}: Inputs: {len(inputs)}, Labels: {len(labels)}")
    if len(inputs) != len(labels):
        raise ValueError(f"Mismatch in batch {i}: {len(inputs)} inputs vs {len(labels)} labels")

In [ ]:
data_iter = data_manager.get_torch_iterator(TRAIN)

# Count the total number of inputs and labels
total_inputs = 0
total_labels = 0

for inputs, labels in data_iter:
    total_inputs += len(inputs)
    total_labels += len(labels)

print(f"Total inputs: {total_inputs}, Total labels: {total_labels}")

In [ ]:
data_iter = data_manager.get_torch_iterator(TRAIN)

# Debugging the iterator
for i, (input_text, label) in enumerate(data_iter):
    print(f"Input {i}: {input_text}, Label: {label}")
    if i == 10:  # Check the first 10 samples
        break

In [ ]:
init_data_manager = True